In [1]:
import torch

In [44]:
tensor_path = "/home/max/Studium/Leipzig/Semster6/Math_and_ML/math-ml/experiments/test_cot_llama3/output_2025-06-25_17-24.pt"

exp_results = torch.load(tensor_path)

In [45]:
exp_results.keys()

dict_keys(['prompt0', 'prompt1', 'prompt2', 'prompt3', 'prompt4', 'prompt5', 'prompt6', 'prompt7', 'prompt8', 'prompt9'])

In [46]:
exp_results['prompt0'].keys()

dict_keys(['generated_tokens', 'decoded_tokens', 'top_p_tokens', 'top_p_logits', 'top_p_probs', 'prompt', 'question', 'ground_truth', 'latency'])

In [56]:
prompt = 'prompt0'
print(f"Answer: {' '.join(exp_results[prompt]['decoded_tokens'])}")
print()
print(exp_results[prompt]['ground_truth'])

Answer: A : She  sells   16  -   3  -   4  =   9  eggs  for  $ 2  each .   9  x   2  =   18  dollars .   18  < eos

Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


In [ ]:
exp_results['prompt0']['top_p_probs']

In [7]:
exp_results['prompt0']['top_p_tokens']

[tensor([   185,     32,    207, 100001,    338,    459,    334,     27,  35829,
           1724,   3132,    363]),
 tensor([    32,    185,     27,  32349, 100001,     48,      3,      7,    549,
             16,     40,     64,   7900,    338,   2819,   1847,   2640,   3539,
           9081,     12,     17,   2960,   1567,    207,  90155,   4622,  95991,
           3673,   2054,    774,  28439,  43778,  78268,     20,     33,     18,
          99297,   4453,   4160,  11912,      9,   6017,     35,     34,   1988,
             19,     21]),
 tensor([25]),
 tensor([   363,     16,     17,    207,     19,     23,     18,     21,     20,
             22,     24,  40804,   8763,     15,    185,  12900,    570,  14957,
           2943,  12034,   4622,    334,     32,   3539,  15718,   1862,      3,
           2130,    338,    628,    549,   9313,   1847, 100001,   1202,   2819,
          10895,    459,    429,   1273,     40,  35829,     35,   2721,    774,
           1724,     33,  28625,

In [8]:
exp_results['prompt0']['generated_tokens']

tensor([185,  32,  25,  23,  27,  68, 378])

In [9]:
len(exp_results['prompt0']['decoded_tokens'])

7

In [30]:
raw_test = 'The answer is 72.0'
test = 72.0

In [28]:
reverse = test[::-1]
print(reverse)

cba


In [34]:
test = str(test)
raw_test_rvs = raw_test[::-1]
for char in test:
    for idx,char_b in enumerate(raw_test_rvs):
        if char == char_b:
            indice = len(raw_test_rvs) - idx -1
            print(indice)
            break

14
15
16
17
